# Part 1

In [27]:
test = '''[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up'''

In [22]:
from collections import namedtuple
import re

Record = namedtuple('Record', 'year month day hour minute message')
line_re = re.compile(r'^\[(\d\d\d\d)-(\d\d)-(\d\d) (\d\d):(\d\d)\] (.*)$')
guard_re = re.compile(r'^Guard #(\d+) begins shift$')

def parse_line(line):
    match = line_re.match(line)
    if not match:
        raise ValueError(f'Cannot parse line: {line}')
    args = list(map(int, match.group(1,2,3,4,5))) + [match.group(6)]
    return Record(*args)

def parse_guard_id(message):
    match = guard_re.match(message)
    if not match:
        raise ValueError(f'Cannot parse message: {message}')
    return int(match.group(1))

In [14]:
print(parse_line('[1518-11-01 00:00] Guard #10 begins shift'))

Record(year=1518, month=11, day=1, hour=0, minute=0, message='Guard #10 begins shift')


In [23]:
parse_guard_id('Guard #10 begins shift')

10

In [52]:
def parse_lines(lines):
    '''
    Each guard is represented as a 60 item list, one for each minute. As
    we parse lines, we keep of which guard is on duty and add increment
    each item where the guard is sleeping.
    '''
    guards = dict()
    current_guard = None
    sleep_starts = None

    for line in lines:
        record = parse_line(line)
        if record.message.startswith('falls'):
            sleep_starts = record.minute
        elif record.message.startswith('wakes'):
            if sleep_starts is None:
                print(record)
                raise Exception("Guard wakes up, but wasn't asleep!")
            try:
                guard = guards[current_guard]
            except KeyError:
                guard = [0] * 60
                guards[current_guard] = guard
            for i in range(sleep_starts, record.minute):
                guard[i] += 1
            sleep_starts = None
        elif record.message.startswith('Guard'):
            current_guard = parse_guard_id(record.message)
        else:
            raise Exception('Unexpected message: {}'.format(record.message))
    
    return guards

In [35]:
test_guards = parse_lines(test.split('\n'))
print(test_guards[10])
print(test_guards[99])

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]


In [34]:
def max_guards(guards):
    ''' Return the ID of the guard who slept the most. '''
    max_id, max_sleep = None, 0
    for id_, guard in guards.items():
        guard_sleep = sum(guard)
        if guard_sleep > max_sleep:
            max_id, max_sleep = id_, guard_sleep
    return max_id, max_sleep

In [36]:
max_guards(test_guards)

(10, 50)

In [65]:
def max_guard_minute(guard):
    ''' Return the minute where this guard slept most frequently. '''
    max_minute, max_val = -1, -1
    for minute, val in enumerate(guard):
        if val > max_val:
            max_minute, max_val = minute, val
    return max_minute, max_val

In [66]:
max_guard_minute(test_guards[10])

(24, 2)

In [48]:
10 * 24

240

In [56]:
with open('input.txt') as input_:
    lines = list(input_)
    lines.sort()
    guards = parse_lines(lines)
len(guards)

20

In [58]:
max_guards(guards)

(641, 498)

In [67]:
max_guard_minute(guards[641])

(41, 15)

In [60]:
641 * 41

26281

# Part 2

In [68]:
def max_all_guards(guards):
    ''' Return the guard/minute with the highest single minute count. '''
    max_id, max_minute, max_val = -1, -1, -1
    for id_, guard in guards.items():
        guard_minute, guard_val = max_guard_minute(guard)
        if guard_val > max_val:
            max_id, max_minute, max_val = id_, guard_minute, guard_val
    return max_id, max_minute, max_val

In [69]:
max_all_guards(guards)

(1973, 37, 18)

In [70]:
1973 * 37

73001